In [2]:
import requests
from bs4 import BeautifulSoup
import io
from PyPDF2 import PdfReader
import re
import unicodedata
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)
os.makedirs("Scraping", exist_ok=True)
pd.set_option('display.max_rows', None)    
pd.set_option('display.max_columns', None)  

# Grados

Obtener ramas y sus urls

In [15]:
def get_urls_de_ramas(url="https://www.comunidad.madrid/servicios/educacion/oferta-grados-oficiales-universidades"):
    data = []

    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    contenedor = soup.find("div", id="carousel-links-gallery-parrafo")

    if contenedor:
        for a in contenedor.find_all("a", href=True):
            nombre_rama = a.get_text(strip=True)   
            enlace = a["href"]                     
            data.append({"Nombre rama": nombre_rama, "URL": enlace})

    df = pd.DataFrame(data)

    return df


In [16]:
tabla_ramas=get_urls_de_ramas()
tabla_ramas.to_csv("Scraping/tabla_ramas.csv", index=False)
tabla_ramas

,Nombre rama,URL
0,Artes y Humanidades,https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-artes-humanidades
1,Ciencias Sociales y Jurídicas,https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-ciencias-sociales-juridicas
2,Ciencias,https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-ciencias
3,Ciencias de la Salud,https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-ciencias-salud
4,Ingeniería y Arquitectura,https://www.comunidad.madrid/servicios/educacion/estudiar-universidad-grados-rama-conocimiento-ingenieria-arquitectura


Obtener urls de las areas

In [17]:
def get_pdfs(tabla_ramas):
    data = []

    for _, row in tabla_ramas.iterrows():
        nombre_rama = row["Nombre rama"]
        url = row["URL"]

        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            fieldsets = soup.find_all(
                "fieldset",
                class_="collapsible group-paragraph-parent-fieldset field-group-fieldset panel panel-default form-wrapper"
            )

            for fieldset in fieldsets:
                for a in fieldset.find_all("a", href=True):
                    href = a["href"]
                    if href.lower().endswith(".pdf"):
                        if href.startswith("/"):
                            href = "https://www.comunidad.madrid" + href

                        nombre_area = a.get_text(strip=True)
                        nombre_area = nombre_area.replace("\u00a0", " ")

                        nombre_area = re.sub(
                            r"(?i)acc[eé]de a la ficha informativa d[el|e|en]+\s*", "", nombre_area
                        )

                        nombre_area = re.sub(
                            r"(?i)^área\s+(de|en)\s+", "", nombre_area
                        )

                        nombre_area = re.sub(r"\s+", " ", nombre_area).strip()

                        data.append({
                            "Nombre rama": nombre_rama,
                            "Nombre área": nombre_area,
                            "URL": href
                        })

        except Exception as e:
            print(f"Error al procesar {url}: {e}")

    df = pd.DataFrame(data, columns=["Nombre rama", "Nombre área", "URL"])
    return df


In [18]:
tabla_pdfs=get_pdfs(tabla_ramas)
tabla_pdfs.to_csv("Scraping/tabla_pdfs.csv", index=False)
tabla_pdfs

,Nombre rama,Nombre área,URL
0,Artes y Humanidades,Antropología Social y Cultural,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/01_antropologia_2023_24_indiv.pdf
1,Artes y Humanidades,Artes,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/02_artes_2023_24_indiv.pdf
2,Artes y Humanidades,Diseño,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/03_diseno_2023_24_indiv.pdf
3,Artes y Humanidades,Estudios Culturales,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/04_estudios_culturales_2023_24_indiv.pdf
4,Artes y Humanidades,Estudios de Asia y África,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/05_estudios_de_asia_y_africa_2023_24_indiv.pdf
5,Artes y Humanidades,Estudios Semíticos e Islámicos,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/06_estudios_semiticos_e_islamicos_2023_24_indiv.pdf
6,Artes y Humanidades,Filosofía,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/07_filosofia_2023_24_indiv.pdf
7,Artes y Humanidades,Geografía e Historia,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/08_geografia_e_historia_2023_24_indiv.pdf
8,Artes y Humanidades,Humanidades,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/09_humanidades_2023_24_indiv.pdf
9,Artes y Humanidades,Lengua y Literatura Española,https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/10_lengua_y_literatura_espanola_2023_24_indiv.pdf


Obtener informacion (texto bruto) de cada grado

In [19]:
pdfs = tabla_pdfs["URL"].tolist()#[:3] 

resultados = []

for idx, url in enumerate(pdfs):
    try:
        rama = tabla_pdfs.loc[idx, "Nombre rama"]
        area = tabla_pdfs.loc[idx, "Nombre área"] if "Nombre área" in tabla_pdfs.columns else "(no definido)"

        response = requests.get(url)
        response.raise_for_status()
        pdf_file = io.BytesIO(response.content)

        reader = PdfReader(pdf_file)
        num_pages = len(reader.pages)

        bloque_activo = False
        texto_bloque = ""

        for i in range(num_pages):
            pagina = reader.pages[i]
            texto_pagina = pagina.extract_text() or ""

            # Limpiar el texto
            texto_pagina = texto_pagina.replace("\n", " ").replace("-", "")
            texto_pagina = re.sub(r"\s+", " ", texto_pagina)
            texto_pagina = texto_pagina.replace("\u00a0", " ")

            # Revisar si contiene el string de inicio
            if "Descripción de la titulación" in texto_pagina:
                if bloque_activo:
                    # Cerrar bloque anterior y guardar fila
                    resultados.append({
                        "rama": rama,
                        "area": area,
                        "url": url,
                        "nombre_grado": "",
                        "descripcion": "",
                        "salidas": "",
                        "centros": "",
                        "texto_bruto": texto_bloque.strip()
                    })
                    # Iniciar nuevo bloque
                    texto_bloque = texto_pagina + " [[[[[[SALTO DE PÁGINA]]]]]]"
                else:
                    # Comenzar primer bloque
                    bloque_activo = True
                    texto_bloque = texto_pagina + " [[[[[[SALTO DE PÁGINA]]]]]]"
            else:
                if bloque_activo:
                    # Agregar página al bloque actual
                    texto_bloque += " " + texto_pagina + " [[[[[[SALTO DE PÁGINA]]]]]]"

        # Al final del documento, si hay bloque activo, guardarlo
        if bloque_activo and texto_bloque.strip():
            resultados.append({
                "rama": rama,
                "area": area,
                "url": url,
                "nombre_grado": "",
                "descripcion": "",
                "salidas": "",
                "centros": "",
                "texto_bruto": texto_bloque.strip()
            })

    except Exception as e:
        print(f"Error procesando {url}: {e}")
        resultados.append({
            "rama": rama,
            "area": area,
            "url": url,
            "nombre_grado": "(error)",
            "descripcion": "(error)",
            "salidas": "(error)",
            "centros": "(error)",
            "texto_bruto": "(error)"
        })

# Crear DataFrame final
df_pdf_info = pd.DataFrame(resultados)
df_pdf_info.to_csv("Scraping/tabla_texto_bruto.csv", index=False)


KeyboardInterrupt: 

Obtener información segmentada de cada grado

In [ ]:
for idx, row in df_pdf_info.iterrows():
    texto = row["texto_bruto"]
    
    # Tomar solo el texto hasta el primer [SALTO DE PÁGINA]
    partes = texto.split("[SALTO DE PÁGINA]")
    primer_bloque = partes[0].strip()
    
    # Inicializamos los valores
    nombre_grado, descripcion, salidas, centros = "(caso no contemplado)", "(caso no contemplado)", "(caso no contemplado)", "(caso no contemplado)"
    
# ------------------ NOMBRE GRADO ------------------

    # Descripción y salidas en la primera página
    if "Descripción de la titulación" in primer_bloque and "Salidas profesionales" in primer_bloque:
        
        matches = list(re.finditer(r"Grado en", primer_bloque))

        if matches:
            # Última aparición
            last_match = matches[-1]
            start_pos = last_match.start()
            
            # Desde ahí hasta primer número o corchete
            resto_texto = primer_bloque[start_pos:]
            match_final = re.match(r"(Grado en\s+.*?)(?=\d|\[)", resto_texto, re.DOTALL)
            
            if match_final:
                nombre_grado = match_final.group(1).strip()
            else:
                nombre_grado = "(caso no contemplado)"
        else:
            nombre_grado = "(caso no contemplado)"

    else:
        # Buscar en el texto antes del primer salto de página si hay "Grado en" en los 100 caracteres previos
        pos_salto = texto.find("[SALTO DE PÁGINA]")
        if pos_salto != -1:
            antes_salto = texto[:pos_salto]
            match_grado = re.search(r"Grado en.{0,100}$", antes_salto, re.DOTALL)
            if match_grado:
                # Si se encuentra, tomar desde esa posición hasta el salto de página
                start_pos = match_grado.start()
                nombre_grado = texto[start_pos:pos_salto].strip()
            else:
                nombre_grado = "(caso no contemplado)"
        else:
            nombre_grado = "(caso no contemplado)"


# ------------------ DESCRIPCION ------------------

    if "Descripción de la titulación" in primer_bloque:
        if "Salidas profesionales" in primer_bloque:
            # Caso normal: entre ambos
            match_desc = re.search(
                r"Descripción de la titulación(.*?)Salidas profesionales",
                primer_bloque,
                re.DOTALL
            )
            if match_desc:
                descripcion = match_desc.group(1).strip()
        else:
            # Caso alternativo: no aparece "Salidas profesionales" en el primer bloque
            match_desc = re.search(
                r"Descripción de la titulación(.*?)(?=Salidas profesionales|$)",
                texto,
                re.DOTALL
            )
            if match_desc:
                descripcion = match_desc.group(1).strip()

    if nombre_grado != "(caso no contemplado)" and descripcion != "(caso no contemplado)":
        ultimos_50 = descripcion[-len(nombre_grado)*2:]  
        pos_grado = ultimos_50.find(nombre_grado)
        if pos_grado != -1:
            # Calcular posición en el string completo
            pos_final = len(descripcion) - len(nombre_grado)*2 + pos_grado
            # Cortar salidas antes del nombre_grado
            descripcion = descripcion[:pos_final].rstrip()

        for match in re.finditer(r"\[+SALTO DE PÁGINA\]+", descripcion):
            pos_salto = match.start()  # posición del salto
            inicio_check = max(0, pos_salto - len(nombre_grado)*2)  # mirar 100 caracteres antes
            fragmento_previo = descripcion[inicio_check:pos_salto]

            # Buscar nombre_grado en los últimos 50 caracteres
            pos_grado = fragmento_previo.find(nombre_grado)
            if pos_grado != -1:
                # Calcular posición absoluta del inicio del grado
                pos_abs_grado = inicio_check + pos_grado
                # Eliminar desde nombre_grado hasta el salto (sin incluir el salto)
                descripcion = descripcion[:pos_abs_grado].rstrip() + descripcion[pos_salto:]
            
        patron = rf"Grado en(.{{0,{len(nombre_grado)*4}}})\d+"
        match = re.search(patron, descripcion, re.DOTALL)
        if match:
            start_pos = match.start()  # posición donde empieza "Grado en"
            end_pos = match.end()      # posición después del número
            # Eliminar desde "Grado en" hasta el número (inclusive)
            descripcion = descripcion[:start_pos].rstrip() + descripcion[end_pos:].lstrip()



    # Eliminar saltos de página internos
    descripcion = re.sub(r"\[+SALTO DE PÁGINA\]+", " ", descripcion)


    # ------------------ SALIDAS ------------------

    if "Salidas profesionales" in texto:
        start_pos = texto.find("Salidas profesionales") + len("Salidas profesionales")
        
        # Buscar primer salto de página después de Salidas profesionales
        pos_salto = texto.find("[[[[[[SALTO DE PÁGINA]]]]]]", start_pos)
        
        # Determinar límite final
        if pos_salto != -1:
            post_salto = texto[pos_salto + len("[[[[[[SALTO DE PÁGINA]]]]]]"):
                            pos_salto + len("[[[[[[SALTO DE PÁGINA]]]]]]") + 10]
            
            if "Más info" in post_salto or "Direcciones" in post_salto:
                # Cortar en nombre_grado, sin incluirlo
                if nombre_grado != "(caso no contemplado)":
                    pos_grado = texto.find(nombre_grado, start_pos)
                    end_pos = pos_grado if pos_grado != -1 else len(texto)
                else:
                    # Si no hay nombre_grado definido, cortar en salto de página
                    end_pos = pos_salto
            else:
                # Cortar en "Direcciones de los centros donde se imparte la titulación"
                end_pos = texto.find("Direcciones de los centros", start_pos)
                if end_pos == -1:
                    end_pos = len(texto)
        else:
            # No hay salto de página, cortar en "Direcciones de los centros..."
            end_pos = texto.find("Direcciones de los centros", start_pos)
            if end_pos == -1:
                end_pos = len(texto)
        
        # Extraer salidas
        salidas = texto[start_pos:end_pos].strip()
        

        
        # Limpiar espacios múltiples
        salidas = re.sub(r"\s+", " ", salidas)
    else:
        salidas = "(caso no contemplado)"

    
    if nombre_grado != "(caso no contemplado)" and salidas != "(caso no contemplado)":
        ultimos_50 = salidas[-len(nombre_grado)*2:]  
        pos_grado = ultimos_50.find(nombre_grado)
        if pos_grado != -1:
            # Calcular posición en el string completo
            pos_final = len(salidas) - len(nombre_grado)*2 + pos_grado
            # Cortar salidas antes del nombre_grado
            salidas = salidas[:pos_final].rstrip()

        for match in re.finditer(r"\[+SALTO DE PÁGINA\]+", salidas):
            pos_salto = match.start()  # posición del salto
            inicio_check = max(0, pos_salto - len(nombre_grado)*2)  # mirar 100 caracteres antes
            fragmento_previo = salidas[inicio_check:pos_salto]

            # Buscar nombre_grado en los últimos 50 caracteres
            pos_grado = fragmento_previo.find(nombre_grado)
            if pos_grado != -1:
                # Calcular posición absoluta del inicio del grado
                pos_abs_grado = inicio_check + pos_grado
                # Eliminar desde nombre_grado hasta el salto (sin incluir el salto)
                salidas = salidas[:pos_abs_grado].rstrip() + salidas[pos_salto:]
            
        patron = rf"Grado en(.{{0,{len(nombre_grado)*4}}})\d+"
        match = re.search(patron, salidas, re.DOTALL)
        if match:
            start_pos = match.start()  # posición donde empieza "Grado en"
            end_pos = match.end()      # posición después del número
            # Eliminar desde "Grado en" hasta el número (inclusive)
            salidas = salidas[:start_pos].rstrip() + salidas[end_pos:].lstrip()



    # Eliminar saltos de página internos
    salidas = re.sub(r"\[+SALTO DE PÁGINA\]+", " ", salidas)

# ------------------ NOMBRE GRADO (incrustado en descripcion)------------------

    if nombre_grado== "(caso no contemplado)":
        texto_sacado = ""
        texto = descripcion  

        # Buscar todas las apariciones de "Grado en"
        matches = list(re.finditer(r"\bGrado en\b", texto))

        if len(matches) == 1:
            match = matches[0]
            start_pos = match.start()

            # Obtener carácter anterior (ignorando espacios)
            prev_char_match = re.search(r"\S(?=\s*Grado en)", texto[:start_pos][-10:])
            prev_char = prev_char_match.group(0) if prev_char_match else ""

            # Separar el texto en palabras después de "Grado en"
            palabras = re.findall(r"\b\w+\b", texto[match.start():])
            
            # Caso 1: hay punto antes
            if prev_char == ".":
                # Tomar 20 palabras después de "Grado en"
                segmento = " ".join(palabras[:20])
                # Buscar la última palabra que empieza en mayúscula seguida por dos minúsculas
                patron_final = r"([A-ZÁÉÍÓÚÜÑ][a-záéíóúüñ]+)(?:\s+[a-záéíóúüñ]+){2}"
                match_final = list(re.finditer(patron_final, segmento))
                if match_final:
                    ultima = match_final[-1].group(1)
                    fin_pos = texto.find(ultima, start_pos)
                    texto_sacado = texto[start_pos:fin_pos].strip()
                    texto = texto[:start_pos] + texto[fin_pos:]
            else:
                # Caso 2: no hay punto delante → tomar 15 palabras después de "Grado en"
                segmento = " ".join(palabras[:15])
                # Buscar la última palabra que empieza en mayúscula
                match_final = list(re.finditer(r"[A-ZÁÉÍÓÚÜÑ][a-záéíóúüñ]+", segmento))
                if match_final:
                    ultima = match_final[-1].group(0)
                    fin_pos = texto.find(ultima, start_pos) + len(ultima)
                    texto_sacado = texto[start_pos:fin_pos].strip()
                    texto = texto[:start_pos] + texto[fin_pos:]

        descripcion=texto
        nombre_grado=texto_sacado



# ------------------ CENTROS ------------------

    # Centros: entre "Direcciones de los centros donde se imparte la titulación" y el final del texto
    match_centros = re.search(
        r"Direcciones\s*de\s*los\s*centros\s*donde\s*se\s*imparte\s*la*(.*)$",
        row["texto_bruto"],
        re.DOTALL | re.IGNORECASE
    )

    if not match_centros:
        # Intento alternativo: por si hay texto justo pegado antes (sin espacios)
        match_centros = re.search(
            r".*?Direcciones\s*de\s*los\s*centros\s*donde\s*se\s*imparte\s*la*(.*)$",
            row["texto_bruto"],
            re.DOTALL | re.IGNORECASE
        )
    
        if not match_centros:
            match_centros = re.search(
                r"Direcciones\s*de\s*los\s*centros\s*donde\s*se\s*imparte\s*el*(.*)$",
                row["texto_bruto"],
                re.DOTALL | re.IGNORECASE
            )

            if not match_centros:
                # Intento alternativo: por si hay texto justo pegado antes (sin espacios)
                match_centros = re.search(
                    r".*?Direcciones\s*de\s*los\s*centros\s*donde\s*se\s*imparte\s*el*(.*)$",
                    row["texto_bruto"],
                    re.DOTALL | re.IGNORECASE
                )

            else:
                centros = "(caso no contemplado)"

    if match_centros:
        centros = match_centros.group(1)
        centros = centros.replace("[[[[[[SALTO DE PÁGINA]]]]]]", " ").strip()
    else:
        centros = "(caso no contemplado)"

    
    if centros !="(caso no contemplado)":
        centros = re.sub(r"\btitulación\b", "", centros, flags=re.IGNORECASE).strip()
    
    if nombre_grado !="(caso no contemplado)":
        nombre_grado = nombre_grado.replace("[", " ").strip()
        nombre_grado = nombre_grado.replace("]", " ").strip()
        nombre_grado = nombre_grado.replace("Grado en ", " ").strip()

        if "Descripción de la titulación" in nombre_grado:
            nombre_grado = nombre_grado.split("Descripción de la titulación")[0].strip()

        if "Salidas profesionales" in nombre_grado:
            nombre_grado = nombre_grado.split("Salidas profesionales")[0].strip()




        
# ------------------ ACTUALIZAR FILAS ------------------

    # Actualizar fila
    df_pdf_info.at[idx, "nombre_grado"] = nombre_grado
    df_pdf_info.at[idx, "descripcion"] = descripcion
    df_pdf_info.at[idx, "salidas"] = salidas
    df_pdf_info.at[idx, "centros"] = centros
    for col in ["descripcion", "salidas"]:
        if col in df_pdf_info.columns:
            df_pdf_info[col] = df_pdf_info[col].astype(str).apply(lambda x: re.sub(r"\d+", "", x).strip())

            


df_pdf_info[["rama", "area", "nombre_grado", "descripcion", "salidas", "centros"]].to_csv("Scraping/tabla_grados.csv", index=False)


In [ ]:
df_pdf_info[["rama", "area", "nombre_grado", "descripcion", "salidas", "centros"]]

,rama,area,nombre_grado,descripcion,salidas,centros
0,Artes y Humanidades,Antropología Social y Cultural,Antropología Social y Cultural,"Esta titulación permite tener un conocimiento y una comprensión de la diversidad de socie dades y culturas humanas, y apreciar la com plejidad y riqueza potenciales, para poder explicar las causas de esta variedad. Conocer y desarrollar el estudio comparado de las so ciedades humanas. Tener un conocimiento detallado de los ámbitos temáticos de la an tropología social y cultural (procesos econó micos y ecológicos; sistemas de parentesco y relaciones de género, estructuras políticas y jurídicas; creencias, religión y rituales, y sis temas simbolicocognitius) en relación con la diversidad cultural y las implicaciones para la comprensión y la resolución de problemas sociales. Conocer la importancia cultural del trabajo de campo empírico como método primario para la recogida de datos para la generación de teorías antropológicas.","Los graduados en esta titulación podrán des empeñar funciones relacionadas con la diver sidad cultural, el patrimonio etnológico, el desarrollo territorial y la cooperación interna cional; Docencia; Investigación básica e in vestigación aplicada a distintos ámbitos; Con tribución al estudio y la intervención en temas sociales candentes de nuestro país, como el multiculturalismo, los nuevos espacios de co municación, los procesos migratorios, o as pectos relacionados con la ciudadanía y la igualdad de género; Mediación intercultural (resolución de conflictos); La participación ciudadana; planificación territorial y diseño de políticas públicas; el desarrollo local, rural y urbano; la gestión de Parques Naturales; la educación secundaria (temática de educación cívica/para la ciudadanía); la coordinación de proyectos en organizaciones no guberna mentales (ONGs); El asesoramiento para la implantación de planes de igualdad de opor tunidades en empresas e instituciones públi cas; la consultoría a medios de comunicación públicos y privados precisan tener en cuenta las leyes de igualdad y violencia de género.","Universidad Nacional de Educación a Distancia UNED (Pública) Facultad de Filosofía Paseo de la Senda del Rey, 7 (Edificio Humanidades del campus sur) 28040 Madrid Tels.: 91 398 69 03 / 05 www.uned.es negociado.alumnos.filosofia@adm.uned.es"
1,Artes y Humanidades,Artes,Bellas Artes,"Los estudios de Bellas Artes tienen entre sus objetivos fundamentales la conservación y expansión del patrimonio artísticocultural, la educación estética y la formación técni cocientífica del alumno en el campo profe sional del arte puro, de la estética aplicada o de la docencia gráficoplástica. El Grado en Bellas Artes supone, para el que la es tudia y la ejerce, un plano de investigación, ensayos y experimentos de las artes plásticas y de las leyes que imperan en el mundo de la creatividad. La carrera de Bellas Artes se orienta a la formación de artistas plásticos en su acepción más amplia. Además, pretende proporcionar una formación teórica, práctica y metodológica en las distintas manifestacio nes que puede adoptar la práctica artística y sus aplicaciones. Las enseñanzas pretenden promover una actitud crítica y creadora a par tir de un amplio conocimiento de las corrien tes del pensamiento estético. En este caso, el objetivo es descubrir nuevas técnicas de pintura, escultura o fotografía. Crear e innovar son los dos grandes valores de esta carrera.","Las profesiones que estos titulados pueden ejercer son muy diversas: pintor, escultor, dibujante, fotógrafo, crítico e historiador de arte, asesor de compras de arte, guía cultural y de museos, gestor de entes relacionados con la cultura y el arte (museos, galerías de arte, fundaciones), restaurador, decorador y docente. También, pueden desempeñar su actividad profesional en el campo editorial, centros de diseño (industrial, textil, mobiliario, editorial, audiovisual, gráfico), agencias de publicidad, centros de enseñanza artística y colegios o

# Notas

In [8]:
def get_pdfs_notas(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    bloques = soup.find_all("div", id=re.compile(r"^text-link-parrafo"))

    pdfs_notas = []

    for bloque in bloques:
        texto_bloque = bloque.get_text(separator=" ", strip=True).lower()
        
        if "notas de acceso" in texto_bloque:
            for a in bloque.find_all("a", href=True):
                href = a["href"]
                if href.lower().endswith(".pdf"):
                    pdfs_notas.append(href)

    return pdfs_notas


In [9]:
pdfs_notas=get_pdfs_notas('https://www.comunidad.madrid/servicios/educacion/hemeroteca-universitaria')

In [10]:
pdfs_notas

['https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/notas_de_corte_a4_2023_24_def.pdf',
 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/notas_de_corte_2022-23_para_web.pdf',
 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/notas_de_corte_2021_22_web.pdf',
 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/af_notas_de_corte_a4_2020_21_web.pdf',
 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/notas_de_corte_2019_20.pdf',
 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/notas_de_corte_a4_2018_19_baja_web.pdf',
 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/publicacionnotasdecorte2017-18_0.pdf',
 'https://www.comunidad.madrid/sites/default/files/doc/educacion/univ/notascorte.publicacion16-17.pdf_0.pdf',
 'https://www.comunidad.madrid/sites/default/files/doc/educacion/notascorte.publicacion15-16_1.pdf',
 'https://www.comunidad.madrid/sites/default